In [1]:
from pyspark import SparkConf,SparkContext
import re, string

In [2]:
stopwords = ["they", "she", "he", "it", "the", "as", "is", "and"]

# =====================================
#  word count specifications:
#
#  replaced all punctuation with whitespaces
#  replaced ['s] with whitespace
#
#  e.g., "what's your name?" --> ['what', 'your', 'name']
# =====================================
         
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

In [3]:
# create spark with necessary configuration
conf = SparkConf ()
sc = SparkContext.getOrCreate(conf = conf)

# read file
rdd = sc.wholeTextFiles("./Data/*/*")

# filter stop words
rdd = rdd.flatMap(lambda x: [(word, x[0]) for word in x[1].lower().replace("'s", " ").translate(translator).split()])\
        .filter(lambda x: x[0] not in stopwords)

# count sum value
rdd = rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b)\
        .map(lambda x: (x[0][0], [(x[0][1], x[1])]))\
        .reduceByKey(lambda a, b: a + b)


In [4]:
rdd.take(1)

[('project',
  [('file:/notebooks/Data/Hugo/Miserables.txt', 92),
   ('file:/notebooks/Data/Hugo/NotreDame_De_Paris.txt', 90),
   ('file:/notebooks/Data/Tolstoy/anna_karenhina.txt', 90),
   ('file:/notebooks/Data/shakespeare/comedies/tempest', 4),
   ('file:/notebooks/Data/shakespeare/comedies/troilusandcressida', 2),
   ('file:/notebooks/Data/shakespeare/comedies/winterstale', 1),
   ('file:/notebooks/Data/shakespeare/histories/2kinghenryiv', 1),
   ('file:/notebooks/Data/Tolstoy/war_and_peace.txt', 100),
   ('file:/notebooks/Data/shakespeare/comedies/allswellthatendswell', 1),
   ('file:/notebooks/Data/shakespeare/comedies/muchadoaboutnothing', 1),
   ('file:/notebooks/Data/shakespeare/glossary', 1),
   ('file:/notebooks/Data/shakespeare/tragedies/antonyandcleopatra', 1),
   ('file:/notebooks/Data/shakespeare/tragedies/hamlet', 1)])]

In [5]:
rdd.coalesce(1).saveAsTextFile('./result')

In [15]:
# s = "what's your name?"
# s.replace("'s", "").translate(translator).split()

['what', 'your', 'name']